In [2]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime

In [5]:
df = pd.read_csv('data/roadnet/rdnet_in_file3.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CASEQTY                      601 non-null    float64
 1   DESTINATIONLOCATIONID        601 non-null    int64  
 2   INVENTTRANSID                601 non-null    object 
 3   ITEMID                       601 non-null    int64  
 4   ORDERNUMBER                  601 non-null    int64  
 5   ORIGINLOCATIONID             601 non-null    object 
 6   PRODUCTNAME                  601 non-null    object 
 7   ROADNETROUTE                 601 non-null    object 
 8   STOPLOCATIONID               601 non-null    int64  
 9   STOPLOCATIONNAME             601 non-null    object 
 10  WEIGHT                       601 non-null    float64
 11  DYNAMICSRETRIEVALSESSIONID   601 non-null    object 
 12  ROADNETROUTEINTERNALROUTEID  601 non-null    object 
 13  APPTID              

In [7]:
df1 = df.drop_duplicates(keep='first')

In [10]:
df1.to_csv('data/roadnet/rdnet_in_file3_nodups.csv', index=False)

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549 entries, 0 to 580
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CASEQTY                      549 non-null    float64
 1   DESTINATIONLOCATIONID        549 non-null    int64  
 2   INVENTTRANSID                549 non-null    object 
 3   ITEMID                       549 non-null    int64  
 4   ORDERNUMBER                  549 non-null    int64  
 5   ORIGINLOCATIONID             549 non-null    object 
 6   PRODUCTNAME                  549 non-null    object 
 7   ROADNETROUTE                 549 non-null    object 
 8   STOPLOCATIONID               549 non-null    int64  
 9   STOPLOCATIONNAME             549 non-null    object 
 10  WEIGHT                       549 non-null    float64
 11  DYNAMICSRETRIEVALSESSIONID   549 non-null    object 
 12  ROADNETROUTEINTERNALROUTEID  549 non-null    object 
 13  APPTID              